<a href="https://colab.research.google.com/github/michelucci/aadl2-code/blob/master/chapter2/Loading_and_saving_weights_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and saving weights in Keras

(C) 2019 Umberto Michelucci

umberto.michelucci@toelt.ai

www.toelt.ai

Official Documentation Link: https://goo.gl/CA3zDT

In [0]:
import os

import tensorflow as tf
from tensorflow import keras

To make training faster let's consider only 5000 digits.

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:5000]
test_labels = test_labels[:5000]

train_images = train_images[:5000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:5000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


Let's build a very easy model for our testing

In [0]:
model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

model.compile(optimizer='adam',
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, period = 1)

model.fit(train_images, train_labels,  epochs = 5,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

training
Train on 5000 samples, validate on 5000 samples
Epoch 1/5
4992/5000 [============================>.] - ETA: 0s - loss: 0.2422 - acc: 0.9367
Epoch 00001: saving model to training/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
5000/5000 [==============================] - 1s 207us/step - loss: 0.2419 - acc: 0.9368 - val_loss: 0.3469 - val_acc: 0.8966
Epoch 2/5
4768/5000 [===========================>..] - ETA: 0s - loss: 0.1710 - acc: 0.9499
Epoch 00002: saving model to training/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
5000/5000 [==============================] - 1s 200us/step - loss: 0.1703 - acc: 0.9500 - val_loss: 0.3063 - val_acc: 0.9038
Epoch 3/5
4928/5000 [============================>.] - ETA: 0s - loss: 0.1221 - acc: 0.9661
Epoch 00003: saving model to training/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
5000/5000 [==============================] - 1s 185us/step - loss: 0.1216 - acc: 0.9662 - val_loss: 0.2662 - va

In [0]:
!ls -al {checkpoint_dir}

total 1612
drwxr-xr-x 2 root root    4096 Jan 28 21:45 .
drwxr-xr-x 1 root root    4096 Jan 28 20:51 ..
-rw-r--r-- 1 root root      71 Jan 28 21:45 checkpoint
-rw-r--r-- 1 root root 1631492 Jan 28 21:45 cp.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root     647 Jan 28 21:45 cp.ckpt.index


Now let's create a completely untrained model (with the same architecture)

In [0]:
model2 = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

model2.compile(optimizer='adam',
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
loss, acc = model2.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

5000/5000 [==============================] - 0s 80us/step
Untrained model, accuracy:  7.58%


As expected we have a random classification (roughly 10%). Now we reload the weights and reevaluate the accuracy.

In [0]:
model2.load_weights(checkpoint_path)
loss,acc = model2.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5000/5000 [==============================] - 0s 54us/step
Restored model, accuracy: 85.96%


# Saving the weights every $n$ epochs

In [0]:
checkpoint_path = "training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


In [0]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    period=1)


In [0]:
model.save_weights(checkpoint_path.format(epoch=0))


Consider using a TensorFlow optimizer from `tf.train`.


In [0]:
model.fit(train_images, train_labels,
          epochs = 10, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)



Epoch 00001: saving model to training/cp-0001.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00002: saving model to training/cp-0002.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00003: saving model to training/cp-0003.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00004: saving model to training/cp-0004.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00005: saving model to training/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00006: saving model to training/cp-0006.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00007: saving model to training/cp-0007.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00008: saving model to training/cp-0008.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00009: saving model to training/cp-0009.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model t

In [0]:

!ls training

checkpoint			  cp-0006.ckpt.data-00000-of-00001
cp-0000.ckpt.data-00000-of-00001  cp-0006.ckpt.index
cp-0000.ckpt.index		  cp-0007.ckpt.data-00000-of-00001
cp-0001.ckpt.data-00000-of-00001  cp-0007.ckpt.index
cp-0001.ckpt.index		  cp-0008.ckpt.data-00000-of-00001
cp-0002.ckpt.data-00000-of-00001  cp-0008.ckpt.index
cp-0002.ckpt.index		  cp-0009.ckpt.data-00000-of-00001
cp-0003.ckpt.data-00000-of-00001  cp-0009.ckpt.index
cp-0003.ckpt.index		  cp-0010.ckpt.data-00000-of-00001
cp-0004.ckpt.data-00000-of-00001  cp-0010.ckpt.index
cp-0004.ckpt.index		  cp.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001  cp.ckpt.index
cp-0005.ckpt.index


In [0]:
latest = tf.train.latest_checkpoint('training')
print(latest)
print(type(latest))

training/cp-0010.ckpt
<class 'str'>


# Save manually the weights

You can save your weights manually, without the need to define a callback. For example

In [0]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model2.load_weights('./checkpoints/my_checkpoint')

loss,acc = model2.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
5000/5000 [==============================] - 0s 61us/step
Restored model, accuracy: 92.98%


In [0]:
!ls ./checkpoints

checkpoint  my_checkpoint.data-00000-of-00001  my_checkpoint.index


# Save the entire model

In [0]:
model.save('my_model.h5')

In [0]:
!ls

checkpoints  my_model.h5  sample_data  training


In [0]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

5000/5000 [==============================] - 0s 77us/step
Restored model, accuracy: 92.88%
